## 현재 실력으로는 혼자서 만들수가 없어 AI프롬프트를 이용해서 하나하나 만들었습니다.

In [ ]:
# ### 🚢 타이타닉 생존자 예측 모델링 프로젝트

# 1. 환경 설정 및 데이터 로드 (Environment Setup & Data Loading)

# 2. 데이터 탐색 (Exploratory Data Analysis, EDA)

# 3. 데이터 전처리 (Data Preprocessing) 

# 4. 피처 엔지니어링 (Feature Engineering)

# 5. 모델링 및 학습 (Modeling & Training)

# 6. 모델 평가 및 튜닝 (Evaluation & Tuning)

# 7. 최종 예측 및 제출 (Prediction & Submission)

### 1.환경 설정 및 데이터 로드 (Environment Setup & Data Loading)

In [ ]:
## 1. 환경 설정 및 데이터 로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 시각화 그래프가 노트북 안에 나타나도록 설정
%matplotlib inline
# 그래프 스타일 설정
sns.set(style="whitegrid")

# 데이터 경로 설정 (현재 노트북 위치의 상위 폴더(../)에 있는 data 폴더 접근)
train_path = '../data/titanic/train.csv'
test_path = '../data/titanic/test.csv'

# 파일 존재 여부 확인 후 불러오기
if os.path.exists(train_path):
    print(f"파일을 찾았습니다: {train_path}")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    print("데이터 로드 성공!")
    
    # 데이터의 처음 5줄 확인
    display(train_df.head())
else:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {train_path}")
    print(f"현재 작업 경로: {os.getcwd()}")

### 2.데이터 탐색 (Exploratory Data Analysis, EDA)

In [ ]:
# 그래프 스타일 설정 (시각화를 위해)
sns.set(style="whitegrid")

# 1. 데이터 정보 확인 (행 개수, 컬럼 타입, 결측치 존재 여부 등)
print("### 1. 데이터 정보 (Info) ###")
train_df.info()

print("\n" + "="*50 + "\n")

# 2. 기술 통계량 확인 (평균, 표준편차, 4분위수 등 - 수치형 데이터)
print("### 2. 기술 통계량 (Describe) ###")
display(train_df.describe())

print("\n" + "="*50 + "\n")

# 3. 결측치 개수 확인
print("### 3. 결측치 개수 (Null Count) ###")
print(train_df.isnull().sum())

In [ ]:
# 1. 타겟 변수 (Survived) 분포 확인
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# 파이 차트 (생존 비율)
train_df['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')

# 카운트 플롯 (생존자 수)
sns.countplot(x='Survived', data=train_df, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

# 2. 성별(Sex)에 따른 생존율
plt.figure(figsize=(10, 5))
sns.barplot(x='Sex', y='Survived', data=train_df)
plt.title('Survival Rate by Sex')
plt.show()

# 3. 객실 등급(Pclass)에 따른 생존율
plt.figure(figsize=(10, 5))
sns.barplot(x='Pclass', y='Survived', data=train_df)
plt.title('Survival Rate by Pclass')
plt.show()

### 3. 데이터 전처리 (Data Preprocessing) -- **중요!**

In [ ]:
## 3. 데이터 전처리 (Data Preprocessing)

# 1. 결측치 처리 (Missing Values)
# Age(나이): 중간값(median)으로 채우기
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())

# Embarked(탑승 항구): 최빈값(가장 많이 나온 값)으로 채우기
embarked_mode = train_df['Embarked'].mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(embarked_mode)
test_df['Embarked'] = test_df['Embarked'].fillna(embarked_mode)

# Fare(운임): 테스트 데이터에 결측치가 1개 있을 수 있으므로 중간값으로 채우기
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())

# Cabin(객실 번호): 결측치가 너무 많으므로(70% 이상) 컬럼 자체를 삭제
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

# 2. 불필요한 컬럼 제거 (Unnecessary Columns)
# Name(이름), Ticket(티켓 번호)는 단순 처리로는 모델에 도움이 적으므로 삭제
# (PassengerId는 학습에 필요 없지만 나중에 확인을 위해 train에서만 제거하거나 인덱스로 활용 가능)
# 여기서는 학습용 데이터셋 정리가 목적이므로 Name, Ticket 먼저 제거
drop_cols = ['Name', 'Ticket']
train_df = train_df.drop(drop_cols, axis=1)
test_df = test_df.drop(drop_cols, axis=1)

# 3. 범주형 데이터 인코딩 (Categorical Encoding)
# 문자열(String)을 숫자(Number)로 변경해야 모델이 이해할 수 있습니다.

# Sex(성별): male -> 0, female -> 1
sex_mapping = {'male': 0, 'female': 1}
train_df['Sex'] = train_df['Sex'].map(sex_mapping)
test_df['Sex'] = test_df['Sex'].map(sex_mapping)

# Embarked(항구): S -> 0, C -> 1, Q -> 2
embarked_mapping = {'S': 0, 'C': 1, 'Q': 2}
train_df['Embarked'] = train_df['Embarked'].map(embarked_mapping)
test_df['Embarked'] = test_df['Embarked'].map(embarked_mapping)

# 전처리 결과 확인
print("전처리 후 데이터 정보:")
train_df.info()
print("\n전처리 후 상위 5개 데이터:")
display(train_df.head())

### 4. 피처 엔지니어링 (Feature Engineering)

In [ ]:
## 4. 피처 엔지니어링 (Feature Engineering)
from sklearn.preprocessing import StandardScaler

# 1. 새로운 피처 생성: 가족 수 (FamilySize)
# 형제자매/배우자(SibSp) + 부모/자녀(Parch) + 나 자신(1)
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# 2. 새로운 피처 생성: 혼자 탑승 여부 (IsAlone)
# 가족 수가 1명이면 혼자 탑승한 것 (1: True, 0: False)
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1

test_df['IsAlone'] = 0
test_df.loc[test_df['FamilySize'] == 1, 'IsAlone'] = 1

# 3. 데이터 스케일링 (Scaling)
# 나이(Age)와 요금(Fare)의 단위 차이가 크므로, 평균 0, 표준편차 1이 되도록 조정합니다.
# (트리 모델에서는 필수는 아니지만, 로지스틱 회귀 등 다른 모델을 위해 권장됩니다.)

scaler = StandardScaler()
scale_cols = ['Age', 'Fare']

# 학습 데이터(train_df)의 통계량을 기준으로 스케일링 학습(fit)
scaler.fit(train_df[scale_cols])

# 학습된 기준으로 데이터 변환(transform)
train_df[scale_cols] = scaler.transform(train_df[scale_cols])
test_df[scale_cols] = scaler.transform(test_df[scale_cols])

# 결과 확인
print("피처 엔지니어링 완료!")
print("\n생성된 파생 변수 및 스케일링 결과 확인:")
display(train_df[['Age', 'Fare', 'FamilySize', 'IsAlone']].head())

### 5. 모델링 및 학습 (Modeling & Training)

In [ ]:
## 5. 모델링 및 학습 (Modeling & Training)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1. 학습 데이터와 정답(Label) 분리
# PassengerId는 예측에 의미가 없으므로 제외, Survived는 정답이므로 분리
X = train_df.drop(['PassengerId', 'Survived'], axis=1)
y = train_df['Survived']

# 2. 학습용(Train)과 검증용(Validation) 데이터 분리
# 전체 데이터의 20%를 검증용으로 따로 떼어놓습니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"검증 데이터 크기: {X_val.shape}")
print("-" * 30)

# 3. 모델 생성 및 학습, 평가

# (1) 의사결정나무 (Decision Tree)
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_val)
print(f"Decision Tree 정확도: {accuracy_score(y_val, dt_pred):.4f}")

# (2) 랜덤 포레스트 (Random Forest)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_val)
print(f"Random Forest 정확도: {accuracy_score(y_val, rf_pred):.4f}")

# (3) 로지스틱 회귀 (Logistic Regression)
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_val)
print(f"Logistic Regression 정확도: {accuracy_score(y_val, lr_pred):.4f}")

### 6.모델 평가 및 튜닝 (Evaluation & Tuning)

In [ ]:
## 6. 모델 평가 및 튜닝 (Evaluation & Tuning)
from sklearn.model_selection import cross_val_score, GridSearchCV

# 1. 교차 검증 (Cross Validation)
# 데이터를 5등분해서 번갈아가며 검증하여, 결과가 우연히 잘 나온 것이 아님을 확인합니다.
# (이전 단계에서 만든 rf_clf 모델 사용)
scores = cross_val_score(rf_clf, X_train, y_train, cv=5)

print("### 교차 검증 결과 ###")
print(f"개별 정확도: {scores}")
print(f"평균 정확도: {np.mean(scores):.4f}")
print("-" * 30)

# 2. 하이퍼파라미터 튜닝 (GridSearchCV)
# Random Forest의 옵션(나무 개수, 깊이 등)을 다양하게 조합해서 가장 좋은 성능을 찾는 과정입니다.
params = {
    'n_estimators': [50, 100, 200],     # 나무의 개수
    'max_depth': [None, 10, 20],        # 나무의 최대 깊이 (너무 깊으면 과적합 가능성)
    'min_samples_leaf': [1, 2, 4],      # 리프 노드에 있어야 하는 최소 샘플 수
    'min_samples_split': [2, 5, 10]     # 노드 분할을 위한 최소 샘플 수
}

# n_jobs=-1은 컴퓨터의 모든 성능을 써서 빨리 계산하라는 뜻입니다.
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), 
                       param_grid=params, 
                       cv=5, 
                       n_jobs=-1)

grid_rf.fit(X_train, y_train)

print("\n### 튜닝 결과 ###")
print(f"최적의 파라미터: {grid_rf.best_params_}")
print(f"최고 정확도 (Best Score): {grid_rf.best_score_:.4f}")

# 3. 최적의 모델로 업데이트
# 튜닝된 가장 좋은 모델을 best_model 변수에 저장합니다.
best_model = grid_rf.best_estimator_

### 7. 최종 예측 및 제출 (Prediction & Submission)

In [ ]:
# XGBoost 모델 생성 및 학습
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# (주의) 만약 X_train, y_train이 정의되지 않았다면 train_test_split 코드도 다시 실행해야 합니다.
# 안전하게 다시 한 번 데이터를 나눕니다.
X = train_df.drop(['PassengerId', 'Survived'], axis=1)
y = train_df['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의 및 학습
xgb_clf = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

print("XGBoost 모델 학습 완료!")

In [23]:
## 7. 최종 예측 및 제출 (Prediction & Submission)

# 1. 테스트 데이터 준비
# 학습할 때 PassengerId는 사용하지 않았으므로, 테스트 데이터에서도 제외해야 모델이 헷갈리지 않습니다.
X_test_final = test_df.drop(['PassengerId'], axis=1)

# 2. 예측 수행 (XGBoost 모델 사용)
final_pred = xgb_clf.predict(X_test_final)

# (만약 Random Forest 튜닝 모델을 쓰고 싶다면 아래 주석을 풀고 실행하세요)
# final_pred = best_model.predict(X_test_final)

# 3. 제출 파일 생성
# PassengerId와 예측된 Survived 값을 합쳐서 데이터프레임을 만듭니다.
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": final_pred
})

# 결과 확인 (상위 5개)
print("제출 데이터 미리보기:")
display(submission.head())

# 4. CSV 파일로 저장
# index=False를 해야 불필요한 인덱스 번호가 저장되지 않습니다.
submission.to_csv('titanic_submission.csv', index=False)

print("\n'submission.csv' 파일이 성공적으로 저장되었습니다!")

제출 데이터 미리보기:


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0



'submission.csv' 파일이 성공적으로 저장되었습니다!
